In [1]:
import pandas as pd
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
census_income = fetch_ucirepo(id=20) 
  
# data (as pandas dataframes) 
X = census_income.data.features 
y = census_income.data.targets 
  
# variable information 
print(census_income.variables) 


              name     role         type      demographic  \
0              age  Feature      Integer              Age   
1        workclass  Feature  Categorical           Income   
2           fnlwgt  Feature      Integer             None   
3        education  Feature  Categorical  Education Level   
4    education-num  Feature      Integer  Education Level   
5   marital-status  Feature  Categorical            Other   
6       occupation  Feature  Categorical            Other   
7     relationship  Feature  Categorical            Other   
8             race  Feature  Categorical             Race   
9              sex  Feature       Binary              Sex   
10    capital-gain  Feature      Integer             None   
11    capital-loss  Feature      Integer             None   
12  hours-per-week  Feature      Integer             None   
13  native-country  Feature  Categorical            Other   
14          income   Target       Binary           Income   

                       

In [59]:
import polars as pl
import numpy as np

def encode(df: pl.DataFrame, columns) -> pl.DataFrame:
    df_encoded = []
    
    for col in columns:
        dummies = df.select(col).to_dummies()
        if dummies.shape[1] > 1:
            dummies = dummies[:, 1:]
        df_encoded.append(dummies)
    for col in df.columns:
        if col not in columns:
            df_encoded.append(df.select(col))
    
    df_encoded = pl.concat(df_encoded, how="horizontal")
    return df_encoded

# Define the predictors
df = pl.from_pandas(X.filter([ "sex", "age", "education"]))
encoded_df = encode(df, ["sex", "education"])

# There are 4 distinct values for y, but two of them can be merged
y_pl = pl.from_pandas(y['income'].replace("<=50K", "<=50K.").replace(">50K", ">50K."))
y_pl = y_pl.to_dummies()[:,0].to_frame() # 0, 1

y_pl_np = np.array(y_pl, dtype=bool) * 2 -1
y_pl_np * encoded_df


array([[  1,   0,   0, ...,   0,   0,  39],
       [  1,   0,   0, ...,   0,   0,  50],
       [  1,   0,   0, ...,   0,   0,  38],
       ...,
       [  1,   0,   0, ...,   0,   0,  38],
       [  1,   0,   0, ...,   0,   0,  44],
       [ -1,   0,   0, ...,   0,   0, -35]])

In [35]:
np.array()

In [44]:
y_pl.to_dummies()[:,0]

income_<=50K.
u8
1
1
1
1
1
…
1
1
1
